파이토치 
- nn.Model 
- 파이토치의 장점 
    - 자유롭고 파이토닉 하고, 연구하기 좋은 프레임워크로 많이 알려져있음. 
    - 자유도 측면에서는 장점이 있다.
    - FROM RESEARCH TO PRODUCTION . 하나하나의 구조를 이해하는데, 큰 이점이있다. 
    - Low-level, Pythonic, Flexibility
    
    - 예를 들어, 케라스는 fit 함수에 수정할 수 있는 것이 한계가 있는데, 파이토치는 그에 비해 로그를 넣는 것 조차도 쉽게 넣을 수 있다.
- 파이토치 모델의 모든 레이어는 nn.Module 클래스를 따른다.

#### 기본적인 예시

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1,20,5)
        self.conv2 = nn.Conv2d(20,20,5)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

#### modules : __init__에서 정의한 또다른 nn.Module

In [8]:
# 모델 자체를 선언하면, initialization 이 된다.
# 그 후, print 하면 , initialization에서 정의한 모델이 표시된다.  == modules
model = MyModel()
print(a)

MyModel(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)


In [7]:
# 그 후, modules를 통해서도, initialization에서 정의한 모델이 표시된다. 
# 하나의 모델을 정의를 해도, 모델의 안에 쓰이는 모델을 모두 알아 볼 수 있다.
# 하나의 모델을 정의하고, 거기에 연결된 파라미터를 들고있는( 웨이트를 가지고 있는) 모델을 알 수 있다.

list(model.modules())

[MyModel(
   (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
   (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
 ),
 Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1)),
 Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))]

#### forward : 이 모델이 호출 되었을 때 실행되는 함수
- 모델이 호출되서, 인풋과 아웃풋을 만들때, 그 체인의 과정을 의미함.
- 체인의 과정을 포워드로 표시.
- 딥러닝의 전파 방향대로 계산하는 과정

In [19]:
import torch
x = torch.rand((1,1,256,256))  # 임의로 들어감. 
y = model(x) # model.forward(x)    # model을 직접 호출하거나. forward()를 해도 같은 값이 나옴 
y.shape

torch.Size([1, 20, 248, 248])

#### nn.Module Family : nn.Module 을 상속받은 모든 클래스의 공통된 특징
- 모든 nn.Module은 child modules 를 가질 수 있다.
- 내 모델을 정의하는 순간, 그 모델에 연결된 모든 module을 확인할 수 있고, 이용할 수있다!!!!!!
- 모듈안에 저장된 파라미터 수치가 어떤 쉐입으로 되어있는지, 그것을 가지고 저장되 weight를 가지고 output을 가지게 해준것
- 학습되지 않는 weight를 가지고 있더라도, 추후에 학습을 시키면서 loss를 발생시키고, loss를 backward로 전파시키면서, 모델의 weight 파라미터를 업데이터 해준다. 그것의 기반을 여기서의 파라미터를 기반으로 한다. 
- nn.Module을 상속하는 것만으로도, 서로 긴밀하게 이어져있다...

- 모든 nn.Module은 forward()함수를 가지게 된다. 
    - 내가 정의한 모델의 forword()를 한번만 실행한 것으로, 그 모델의 forward에 정의된 모듈 각각의 forward()가 실행된다.
    - 즉, 내가 한번만 실행했음에도, 각각 모델 (nn.Conv2d)안에 있는 forward도 실행됨이 당연하다.전체적인 forward()를 연결시키고 실행시키게 만들수 있다.
    

#### Parameters : 모델에 정의되어있는 modules가 가지고 있는 계산에 쓰일 Parameter

In [20]:
state_dic = model.state_dict()
state_dic

OrderedDict([('conv1.weight',
              tensor([[[[-4.8016e-02, -2.0410e-02,  1.8049e-01,  1.6207e-01, -6.5292e-02],
                        [-7.4546e-02,  9.4582e-02, -1.0564e-01, -1.9039e-01,  1.5610e-01],
                        [ 1.0482e-01, -8.4936e-02, -9.1368e-02, -1.0467e-01, -1.9701e-01],
                        [-3.6042e-02, -1.1216e-01,  1.4316e-01, -1.6665e-01, -1.3588e-01],
                        [ 6.5171e-02, -3.1426e-02,  1.1314e-01,  1.5178e-01, -1.7552e-01]]],
              
              
                      [[[-8.9147e-02, -3.4548e-03, -7.8049e-02, -1.9160e-01,  1.9338e-01],
                        [ 1.6267e-01, -1.0813e-01, -1.1084e-01, -1.1872e-01,  4.2506e-02],
                        [ 1.4112e-01, -7.6011e-05, -4.8736e-02, -1.8018e-01,  1.5834e-01],
                        [-1.0052e-01, -1.8809e-02,  3.8191e-02, -2.7584e-02, -2.3961e-02],
                        [ 4.9303e-02, -8.7673e-02,  1.8724e-01, -1.7193e-01,  1.3774e-01]]],
              
           

In [22]:
params = list(model.parameters())
params

[Parameter containing:
 tensor([[[[-4.8016e-02, -2.0410e-02,  1.8049e-01,  1.6207e-01, -6.5292e-02],
           [-7.4546e-02,  9.4582e-02, -1.0564e-01, -1.9039e-01,  1.5610e-01],
           [ 1.0482e-01, -8.4936e-02, -9.1368e-02, -1.0467e-01, -1.9701e-01],
           [-3.6042e-02, -1.1216e-01,  1.4316e-01, -1.6665e-01, -1.3588e-01],
           [ 6.5171e-02, -3.1426e-02,  1.1314e-01,  1.5178e-01, -1.7552e-01]]],
 
 
         [[[-8.9147e-02, -3.4548e-03, -7.8049e-02, -1.9160e-01,  1.9338e-01],
           [ 1.6267e-01, -1.0813e-01, -1.1084e-01, -1.1872e-01,  4.2506e-02],
           [ 1.4112e-01, -7.6011e-05, -4.8736e-02, -1.8018e-01,  1.5834e-01],
           [-1.0052e-01, -1.8809e-02,  3.8191e-02, -2.7584e-02, -2.3961e-02],
           [ 4.9303e-02, -8.7673e-02,  1.8724e-01, -1.7193e-01,  1.3774e-01]]],
 
 
         [[[-1.2029e-02, -6.2139e-02,  1.0524e-01,  5.9164e-02,  1.0036e-01],
           [-1.4168e-01, -1.2314e-01, -1.3863e-01,  1.1837e-02, -1.5806e-01],
           [-1.1238e-01, -1.9